In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd  # всё равно есть в зависимостях DeepPavlov, поэтому начнём подготовку данных с ним

/home/nab/work/DeepPavlov/venv-dp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Мы хотим подать данные на вход готовому tf.estimator.MyEstimator. Для начала вместо сеточки MyEstimator интересующей нас архитектуры, используем premade DNNClassifier. Поскольку наш датасет легко влезает в память, то проще обойтись стандартными input_fn из tf.estimator, которые под капотом использую tf.data API, но позволяют явно не влезать в него.

In [2]:
snips_df = pd.read_csv('train.csv')

In [3]:
snips_df.head()

,text,intents
0,Add another song to the Cita RomГЎntica playli...,AddToPlaylist
1,add clem burke in my playlist Pre-Party R&B Jams,AddToPlaylist
2,Add Live from Aragon Ballroom to Trapeo,AddToPlaylist
3,add Unite and Win to my night out,AddToPlaylist
4,Add track to my Digster Future Hits,AddToPlaylist


На вход MyEstimator в качестве разметки (лейблов) ждёт уже целые числа, хотя есть возможность подать ему готовый словарь, с помощью которого он сможет преобразовать разметку иного вида в числа. Готовых решений для составления таких словарей я не нашёл, поэтому придётся сделать это руками.

In [4]:
label_vocabulary = ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook',
                    'SearchCreativeWork', 'SearchScreeningEvent']

Ещё MyEstimator будет ждать описания данных, которые ему будут кормиться во время обучения и инференса. Для этой цели существуют tf.feature_cloumn, которые по сути можно тоже отнести к tf.data API. Для случая работы с текстами, конечно, могут пригодиться tf.feature_column.embedding_column, но они кажутся слишком низкоуровневыми, и лучше использовать сразу hub.text_embedding_column.

In [5]:
my_feature_columns = [
    hub.text_embedding_column(key='text',  # должно совпадать с ключом фичи из датасета
                              module_spec='https://tfhub.dev/google/universal-sentence-encoder/1', trainable=False),
    ]

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


В данном случае в качестве text_embedding_column я использовал просто модуль для трансформации текстовой строки любой длины в плотный вектор фиксированной размерности. Но ничего не мешает добавить и других feature_columns, которые будут позволять работать с последовательностью векторов или другими неплотными векторами - главное, чтобы MyEstimator умел это делать. Было бы круто иметь Universal Sentence Encoder формата TF Hub для русского языка в DeepPavlov, не правда ли?

In [6]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    snips_df, # целиком весь DataFrame
    y=snips_df.intents, # здесь только Series - весьма сильное ограничение, хотя для нашего датасета подойдёт
    batch_size=128, # батчуем данные здесь, а в параметрах тренировки указываем количество степов по батчам
    num_epochs=1,  # для стандартной input_fn указываем эпохи здесь, но и в tf.data API это параметр скорее данных, чем тренировщика
    shuffle=True  # непонятно shuffle каждую эпоху свой или лишь один раз в начале
)

In [7]:
train_input_fn()

({'text': <tf.Tensor 'random_shuffle_queue_DequeueUpTo:1' shape=(?,) dtype=string>,
  'intents': <tf.Tensor 'random_shuffle_queue_DequeueUpTo:2' shape=(?,) dtype=string>},
 <tf.Tensor 'random_shuffle_queue_DequeueUpTo:3' shape=(?,) dtype=string>)

Аналогично train_input_fn в реальной ситуации придётся писать ещё и evaluate_input_fn и подобные. Для целей нашей демонстрации мы уже определили всё необходимое, чтобы инстанцировать наш MyEstimator...

In [8]:
snips_classifier = tf.estimator.DNNClassifier(hidden_units=[250, 100],
                                              feature_columns=my_feature_columns,
                                              n_classes=7,
                                              label_vocabulary=label_vocabulary)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_04u45nl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5efad39208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


и обучить его.

In [9]:
snips_classifier.train(input_fn=train_input_fn, max_steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhu

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/LinearLayer/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/LinearLayer/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/tanh_layer_0/bias:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/bias
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/global_step:0 from c

В целом похоже на Estimator из scikit-learn, но всё же они разные и сейчас несовместимы, а обёртки для совместимости Google поддерживать не планирует.

In [12]:
snips_classifier.evaluate(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhu

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/LinearLayer/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/LinearLayer/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/tanh_layer_0/bias:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/bias
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SNLI/Classifier/tanh_layer_0/weights:0 from checkpoint b'/tmp/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables' with SNLI/Classifier/tanh_layer_0/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/global_step:0 from c

{'accuracy': 0.8627377,
 'average_loss': 0.39414582,
 'loss': 50.07859,
 'global_step': 100}

Хочется отметить, что используя tf.estimator API мы ни разу не оперировали такими понятиями, как граф и сессия, и это, на мой взгляд, большой плюс для конечных пользователей. Более того, наша hub.text_embedding_column - это один вычислительный граф, а MyEstimator - другой, и tf соединил эти два графа не заставив нас страдать.
Понятно, что если в DeepPavlov будут переиспользуемые MyEstimator и input_fn для популярных датасетов, то на первых порах не придётся заниматься дизайном собственного API, при этом доставив конечным пользователям немалую ценность.